In [1]:
# ──────────────────────────────────────────────────────────────
#          Multi-Agent Mesh Workflow with LangGraph + Groq
#                    (Google Colab Version - Jan 2026)
# ──────────────────────────────────────────────────────────────

# 1. Install required packages
!pip install -q -U langgraph langchain langchain_groq langchain_community tavily-python

# 2. Import libraries
import os
from typing import Annotated, TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# 3. Set up API keys (recommended: use Colab secrets)
from google.colab import userdata

# Option A: Best & secure way - store in Colab Secrets (left sidebar → key icon)
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
except:
    print("Could not load from secrets. Using fallback method...")

# Option B: Manual input (less secure - visible in notebook)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = input("Enter your Groq API key: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = input("Enter your Tavily API key: ")

# Quick check
print("Groq API key loaded:", bool(os.environ.get("GROQ_API_KEY")))
print("Tavily API key loaded:", bool(os.environ.get("TAVILY_API_KEY")))

# 4. Initialize fast Groq LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",    # or "mixtral-8x7b-32768", "llama3-70b-8192", etc.
    temperature=0.15,
    max_tokens=1024,
    max_retries=2
)

# 5. Shared State (mesh communication via messages + next_agent routing)
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "The shared conversation history"]
    next_agent: str                     # Which agent should run next (mesh handoff)

# 6. Tools - shared across agents
search = TavilySearchResults(
    max_results=3,
    description="Search the web for current information"
)
tools = [search]
tool_node = ToolNode(tools)

# 7. Helper to create agent nodes (they decide next_agent at the end)
def create_agent_node(llm, system_prompt, tools=None):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt + """

You are part of a collaborative multi-agent team.
Use the shared message history to collaborate.
After your main response, **ALWAYS** end with one of these exactly:
next_agent: researcher
next_agent: writer
next_agent: critic
next_agent: END
        """),
        MessagesPlaceholder(variable_name="messages"),
    ])

    if tools:
        chain = prompt | llm.bind_tools(tools)
    else:
        chain = prompt | llm

    def agent_node(state: AgentState):
        messages = state["messages"]
        response = chain.invoke({"messages": messages})

        # Extract next_agent from the very end of response
        content = response.content
        next_agent = "END"
        if "next_agent:" in content:
            next_agent_part = content.split("next_agent:")[-1].strip()
            if next_agent_part in ["researcher", "writer", "critic", "END"]:
                next_agent = next_agent_part
                # Optional: clean response content (remove instruction part)
                content = content.split("next_agent:")[0].strip()
                response = AIMessage(content=content)

        return {
            "messages": messages + [response],
            "next_agent": next_agent
        }

    return agent_node

# 8. Define the three collaborating agents
researcher = create_agent_node(
    llm,
    "You are the Researcher. Gather facts, use search tool when needed, provide evidence.",
    tools=tools
)

writer = create_agent_node(
    llm,
    "You are the Writer. Synthesize, summarize and write clear, well-structured answers."
)

critic = create_agent_node(
    llm,
    "You are the Critic. Review quality, accuracy, completeness. Suggest improvements or hand back."
)

# 9. Router for mesh-style handoffs
def router(state: AgentState) -> str:
    next_agent = state.get("next_agent", "END").strip()
    if next_agent == "researcher":
        return "researcher"
    if next_agent == "writer":
        return "writer"
    if next_agent == "critic":
        return "critic"
    return END

# 10. Build the graph (mesh communication pattern)
workflow = StateGraph(state_schema=AgentState)

workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)
workflow.add_node("tools", tool_node)

# Entry point → researcher starts
workflow.add_edge(START, "researcher")

# Conditional routing after each agent
for agent in ["researcher", "writer", "critic"]:
    workflow.add_conditional_edges(
        agent,
        router,
        {
            "researcher": "researcher",
            "writer": "writer",
            "critic": "critic",
            END: END
        }
    )

# Tools → back to the agent who called them
workflow.add_edge("tools", "researcher")  # for simplicity; can be dynamic if needed

# Compile the graph
graph = workflow.compile()

# ──────────────────────────────────────────────────────────────
#                          RUN THE AGENTS!
# ──────────────────────────────────────────────────────────────

from IPython.display import Markdown, display

query = "What are the latest trends in multi-agent AI systems as of January 2026?"

print(f"\nStarting mesh multi-agent workflow for:\n{query}\n")

initial_state = {
    "messages": [HumanMessage(content=query)],
    "next_agent": "researcher"
}

final_state = graph.invoke(initial_state)

# Show final answer
final_message = final_state["messages"][-1]
print("\n" + "="*60 + "\nFinal Output:\n" + "="*60)
display(Markdown(final_message.content))

# Optional: Show the full conversation trace
print("\n" + "="*60 + "\nFull Conversation Trace:\n" + "="*60)
for msg in final_state["messages"]:
    if isinstance(msg, HumanMessage):
        print(f"\n🧑 Human: {msg.content}")
    elif isinstance(msg, AIMessage):
        role = "🤖 Agent"
        if "tool_calls" in msg.additional_kwargs and msg.additional_kwargs["tool_calls"]:
            role = "🔧 Tool Call"
        print(f"\n{role}: {msg.content.strip()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
Groq API key loaded: True
Tavily API key loaded: True


/tmp/ipython-input-507731880.py:53: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(



Starting mesh multi-agent workflow for:
What are the latest trends in multi-agent AI systems as of January 2026?


Final Output:



Full Conversation Trace:

🧑 Human: What are the latest trends in multi-agent AI systems as of January 2026?

🔧 Tool Call: 


In [2]:
# =============================================================================
#     FIXED & HARDENED: Multi-Agent Mesh with LangGraph + Groq
#     Much better loop prevention - Jan 2026 version
# =============================================================================

# 1. Install packages
!pip install -q -U langgraph langchain langchain_groq langchain-tavily

# 2. Imports
import os
from typing import Annotated, TypedDict, List, Optional
from IPython.display import display, Markdown, clear_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.errors import GraphRecursionError

# 3. API Keys — use secrets if possible
from google.colab import userdata

for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    try:
        os.environ[key] = userdata.get(key)
    except:
        pass

# Fallback
for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    if key not in os.environ or not os.environ[key]:
        os.environ[key] = input(f"Enter {key}: ").strip()

print("Groq API key loaded:", bool(os.environ.get("GROQ_API_KEY")))
print("Tavily API key loaded:", bool(os.environ.get("TAVILY_API_KEY")))

# 4. LLM — very low temperature + strong model for better discipline
llm = ChatGroq(
    model="llama-3.3-70b-versatile",   # or "mixtral-8x7b-32768" if you prefer
    temperature=0.07,                  # ← very important for following instructions
    max_tokens=1400,
    max_retries=3
)

# 5. State with safety counter
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "Shared history"]
    next_agent: str
    tool_call_count: int                # safety counter

# 6. Tools
search = TavilySearch(
    max_results=5,
    description="Search the internet for current and factual information. Use when you need external knowledge."
)
tools = [search]
tool_node = ToolNode(tools)

# 7. Agent node factory — VERY STRICT prompt
def create_agent_node(llm, system_role: str, tools=None):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""\
You are {system_role}.

STRICT RULES - YOU **MUST** FOLLOW THEM EXACTLY OR THE SYSTEM WILL BREAK:

1. First, think step by step and write your complete response / analysis.
2. After your full response, you **MUST** finish with EXACTLY one of these lines:
   next_agent: researcher
   next_agent: writer
   next_agent: critic
   next_agent: END

   Nothing else after this line. No extra text, no dot, no space before "next_agent".

3. Rules for choosing next_agent:
   - If you still need more research → researcher
   - If you have enough info and want to summarize/write nicely → writer
   - If the current answer needs review/improvement → critic
   - When you think the answer is complete and good → END

4. Researcher special rule:
   - Use search tool ONLY when really necessary.
   - After getting results, DO NOT call search again unless previous results were clearly useless.
   - Summarize what you learned and hand off (usually to writer).

Examples of correct ending:
"Summary of findings... next_agent: writer"
"Final polished answer. next_agent: END"
"""),
        MessagesPlaceholder("messages"),
    ])

    chain = prompt | (llm.bind_tools(tools) if tools else llm)

    def node(state: AgentState):
        messages = state["messages"]
        response = chain.invoke({"messages": messages})

        content = response.content.strip()
        next_agent = "END"
        main_content = content

        # Try to extract next_agent robustly
        if "next_agent:" in content.lower():
            parts = content.lower().split("next_agent:")
            main_content = parts[0].strip()
            candidate = parts[-1].strip().split()[0]  # take first word after :
            if candidate in ["researcher", "writer", "critic", "end"]:
                next_agent = candidate

        new_msg = AIMessage(content=main_content)

        return {
            "messages": messages + [new_msg],
            "next_agent": next_agent,
            "tool_call_count": state.get("tool_call_count", 0) + (1 if response.tool_calls else 0)
        }

    return node

# 8. Agents
researcher = create_agent_node(
    llm,
    "Researcher — gather facts, cite sources, use search when needed",
    tools=tools
)

writer = create_agent_node(
    llm,
    "Writer — create clear, structured, well-written final answers"
)

critic = create_agent_node(
    llm,
    "Critic — review quality, accuracy, clarity. Suggest improvements or approve"
)

# 9. Router with safety
def router(state: AgentState) -> str:
    # Safety: too many tool calls → force to writer or end
    if state.get("tool_call_count", 0) >= 6:
        return "writer"

    nxt = state.get("next_agent", "END").lower().strip()
    if nxt in ["researcher", "writer", "critic"]:
        return nxt
    return END

# 10. Build graph
workflow = StateGraph(state_schema=AgentState)

workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "researcher")

for agent in ["researcher", "writer", "critic"]:
    workflow.add_conditional_edges(
        agent,
        router,
        {"researcher": "researcher", "writer": "writer", "critic": "critic", END: END}
    )

workflow.add_edge("tools", "researcher")

graph = workflow.compile()

# =============================================================================
#                         INTERACTIVE CHAT INTERFACE
# =============================================================================

print("\n" + "═"*85)
print("   Multi-Agent Mesh Chat  (Researcher → Writer → Critic)")
print("   Much more stable loop prevention - type 'exit'/'quit' to stop")
print("═"*85 + "\n")

conversation_history: List[BaseMessage] = []

while True:
    query = input("\nYou: ").strip()

    if query.lower() in ["exit", "quit", "q", ""]:
        print("\nGoodbye! ✌️")
        break

    clear_output(wait=True)
    print("\n" + "═"*85)
    print(f"Processing: {query}")
    print("═"*85 + "\n")

    current_messages = conversation_history + [HumanMessage(content=query)]

    try:
        final_state = graph.invoke(
            {
                "messages": current_messages,
                "next_agent": "researcher",
                "tool_call_count": 0
            },
            config={"recursion_limit": 22}   # ← crucial safety limit
        )

        conversation_history = final_state["messages"]

        final_msg = final_state["messages"][-1]
        display(Markdown(f"**Final Answer**\n\n{final_msg.content.strip()}\n"))

    except GraphRecursionError:
        display(Markdown(
            "**Warning**: Agent reached maximum steps (possible loop).\n"
            "Showing last message as result:"
        ))
        if "messages" in locals() and final_state and "messages" in final_state:
            display(Markdown(final_state["messages"][-1].content))
        else:
            display(Markdown("No usable result due to recursion limit. Try rephrasing."))

    # Show short trace
    print("\nLast 4–5 messages (debug trace):")
    for msg in final_state.get("messages", [])[-5:]:
        if isinstance(msg, HumanMessage):
            print(f"You: {msg.content[:90]}...")
        elif isinstance(msg, AIMessage):
            prefix = "🔧 Tool call → " if msg.tool_calls else "🤖 "
            print(f"{prefix}{msg.content[:90]}...")
    print("═"*85)


═════════════════════════════════════════════════════════════════════════════════════
Processing: 1. What are the main differences between classical RL and quantum RL in 2026?
═════════════════════════════════════════════════════════════════════════════════════



**Final Answer**





Last 4–5 messages (debug trace):
You: Hello help in quantum and reinforcement learning...
🤖 To provide assistance in quantum and reinforcement learning, I'll need to break down the t...
You: 1. What are the main differences between classical RL and quantum RL in 2026?...
🤖 ...
═════════════════════════════════════════════════════════════════════════════════════

You: q

Goodbye! ✌️


In [3]:
# =============================================================================
#  Multi-Agent Collaboration with shared state (mesh style)
#  Groq + LangGraph - more stable version (Jan 2026)
# =============================================================================

# ─── 1. Install packages ─────────────────────────────────────────────────────
!pip install -q -U \
    langgraph \
    langchain \
    langchain-groq \
    langchain-tavily

# ─── 2. Imports ──────────────────────────────────────────────────────────────
import os
from typing import Annotated, TypedDict, List

from IPython.display import display, Markdown, clear_output

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.errors import GraphRecursionError

# ─── 3. API Keys ─────────────────────────────────────────────────────────────
from google.colab import userdata

for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    try:
        os.environ[key] = userdata.get(key)
    except:
        pass

# Fallback prompt
for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        os.environ[key] = input(f"→ Enter {key}: ").strip()

print("Groq key   :", "✓" if os.environ.get("GROQ_API_KEY") else "✗")
print("Tavily key :", "✓" if os.environ.get("TAVILY_API_KEY") else "✗")

# ─── 4. LLM ──────────────────────────────────────────────────────────────────
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,           # low temperature → more consistent format following
    max_tokens=1400,
    max_retries=3
)

# ─── 5. State ────────────────────────────────────────────────────────────────
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "shared message history"]
    next: str                    # who should act next
    tool_use_count: int          # simple safety counter

# ─── 6. Tools ────────────────────────────────────────────────────────────────
search = TavilySearch(max_results=4)
tools = [search]
tool_node = ToolNode(tools)

# ─── 7. Agent node factory ───────────────────────────────────────────────────
def make_agent(
    role: str,
    description: str,
    tools=None
):
    system = f"""\
You are the {role} agent in a collaborative team.

{description}

CORE RULES — YOU MUST FOLLOW THEM STRICTLY:

1. Think step by step, then give your answer.
2. At the VERY END of your response write EXACTLY one line:

   next: researcher
   next: writer
   next: critic
   next: END

   Nothing after this line. No dot, no space before "next:", no extra text.

3. Researcher special rule:
   • Use search only when you really need external/current information
   • After getting results — usually summarize & hand over (don't loop forever)

Allowed next values: researcher | writer | critic | END
"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        MessagesPlaceholder("messages"),
    ])

    chain = prompt | (llm.bind_tools(tools) if tools else llm)

    def agent_node(state: AgentState):
        messages = state["messages"]
        response = chain.invoke({"messages": messages})

        content = response.content.strip()
        next_agent = "END"

        # Try to find next: marker (case insensitive, tolerant to spaces)
        lines = content.split("\n")
        for line in reversed(lines):  # look from the end
            line = line.strip().lower()
            if line.startswith("next:"):
                candidate = line[5:].strip()
                if candidate in ["researcher", "writer", "critic", "end"]:
                    next_agent = candidate
                    # remove the instruction line from visible content
                    content = "\n".join(lines[:lines.index(line)+1*-1]).rstrip()
                    break

        msg = AIMessage(content=content)

        return {
            "messages": messages + [msg],
            "next": next_agent,
            "tool_use_count": state.get("tool_use_count", 0) + (1 if response.tool_calls else 0)
        }

    return agent_node

# ─── 8. Create agents ────────────────────────────────────────────────────────
researcher = make_agent(
    role="Researcher",
    description="Gather facts, use search tool when needed, provide evidence.",
    tools=tools
)

writer = make_agent(
    role="Writer",
    description="Create clear, well-structured, concise answers from shared information."
)

critic = make_agent(
    role="Critic",
    description="Review quality, accuracy, clarity. Suggest improvements or approve."
)

# ─── 9. Router with safety ───────────────────────────────────────────────────
def route(state: AgentState) -> str:
    if state.get("tool_use_count", 0) >= 7:
        return "writer"  # force exit from endless research

    nxt = state.get("next", "END").strip().lower()
    if nxt in ["researcher", "writer", "critic"]:
        return nxt
    return END

# ─── 10. Build graph ─────────────────────────────────────────────────────────
workflow = StateGraph(state_schema=AgentState)

workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "researcher")

for node in ["researcher", "writer", "critic"]:
    workflow.add_conditional_edges(
        node,
        route,
        {"researcher": "researcher", "writer": "writer", "critic": "critic", END: END}
    )

workflow.add_edge("tools", "researcher")  # most tools used by researcher

graph = workflow.compile()

# ─── 11. Interactive loop ────────────────────────────────────────────────────
print("\n" + "═"*80)
print("  Multi-Agent mesh chat  (type 'exit' / 'quit' / empty line to stop)")
print("═"*80 + "\n")

history: List[BaseMessage] = []

while True:
    q = input("You: ").strip()

    if not q or q.lower() in ["exit", "quit", "q"]:
        print("\nBye! ✌️")
        break

    clear_output(wait=True)
    print("\n" + "═"*80)
    print(f"Query: {q}")
    print("═"*80 + "\n")

    try:
        result = graph.invoke(
            {
                "messages": history + [HumanMessage(content=q)],
                "next": "researcher",
                "tool_use_count": 0
            },
            config={"recursion_limit": 24}   # hard safety limit
        )

        history = result["messages"]

        # Show final answer
        last = result["messages"][-1]
        display(Markdown(f"**Final answer**\n\n{last.content.strip()}\n"))

    except GraphRecursionError:
        display(Markdown(
            "**Warning** — maximum steps reached (possible loop).\n"
            "Showing last message:"
        ))
        if "messages" in result:
            display(Markdown(result["messages"][-1].content))

    # ── Trace last messages ──────────────────────────────────────────────────
    print("\nRecent messages (debug trace):")
    for m in result["messages"][-6:]:
        if isinstance(m, HumanMessage):
            print(f"You: {m.content[:70]}...")
        elif isinstance(m, AIMessage):
            prefix = "🔧 " if m.tool_calls else "→ "
            print(f"{prefix}{m.content[:70]}...")
    print("═"*80)


════════════════════════════════════════════════════════════════════════════════
Query: hi
════════════════════════════════════════════════════════════════════════════════



**Final answer**





Recent messages (debug trace):
You: help me in quantum rl latets papers and research...
→ ...
You: hi...
→ ...
════════════════════════════════════════════════════════════════════════════════


KeyboardInterrupt: Interrupted by user

In [4]:
# =============================================================================
#  Multi-Agent Collaboration with shared state (mesh style)
#  Groq + LangGraph - more stable version (Jan 2026)
# =============================================================================

# ─── 1. Install packages ─────────────────────────────────────────────────────
!pip install -q -U \
    langgraph \
    langchain \
    langchain-groq \
    langchain-tavily

# ─── 2. Imports ──────────────────────────────────────────────────────────────
import os
from typing import Annotated, TypedDict, List

from IPython.display import display, Markdown, clear_output

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.errors import GraphRecursionError

# ─── 3. API Keys ─────────────────────────────────────────────────────────────
from google.colab import userdata

for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    try:
        os.environ[key] = userdata.get(key)
    except:
        pass

# Fallback prompt
for key in ["GROQ_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        os.environ[key] = input(f"→ Enter {key}: ").strip()

print("Groq key   :", "✓" if os.environ.get("GROQ_API_KEY") else "✗")
print("Tavily key :", "✓" if os.environ.get("TAVILY_API_KEY") else "✗")

# ─── 4. LLM ──────────────────────────────────────────────────────────────────
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,           # low temperature → more consistent format following
    max_tokens=1400,
    max_retries=3
)

# ─── 5. State ────────────────────────────────────────────────────────────────
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "shared message history"]
    next: str                    # who should act next
    tool_use_count: int          # simple safety counter

# ─── 6. Tools ────────────────────────────────────────────────────────────────
search = TavilySearch(max_results=4)
tools = [search]
tool_node = ToolNode(tools)

# ─── 7. Agent node factory ───────────────────────────────────────────────────
def make_agent(
    role: str,
    description: str,
    tools=None
):
    system = f"""\
You are the {role} agent in a collaborative team.

{description}

CORE RULES — YOU MUST FOLLOW THEM STRICTLY:

1. Think step by step, then give your answer.
2. At the VERY END of your response write EXACTLY one line:

   next: researcher
   next: writer
   next: critic
   next: END

   Nothing after this line. No dot, no space before "next:", no extra text.

3. Researcher special rule:
   • Use search only when you really need external/current information
   • After getting results — usually summarize & hand over (don't loop forever)

Allowed next values: researcher | writer | critic | END
"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        MessagesPlaceholder("messages"),
    ])

    chain = prompt | (llm.bind_tools(tools) if tools else llm)

    def agent_node(state: AgentState):
        messages = state["messages"]
        response = chain.invoke({"messages": messages})

        content = response.content.strip()
        next_agent = "END"

        # Try to find next: marker (case insensitive, tolerant to spaces)
        lines = content.split("\n")
        for line in reversed(lines):  # look from the end
            line = line.strip().lower()
            if line.startswith("next:"):
                candidate = line[5:].strip()
                if candidate in ["researcher", "writer", "critic", "end"]:
                    next_agent = candidate
                    # remove the instruction line from visible content
                    content = "\n".join(lines[:lines.index(line)+1*-1]).rstrip()
                    break

        msg = AIMessage(content=content)

        return {
            "messages": messages + [msg],
            "next": next_agent,
            "tool_use_count": state.get("tool_use_count", 0) + (1 if response.tool_calls else 0)
        }

    return agent_node

# ─── 8. Create agents ────────────────────────────────────────────────────────
researcher = make_agent(
    role="Researcher",
    description="Gather facts, use search tool when needed, provide evidence.",
    tools=tools
)

writer = make_agent(
    role="Writer",
    description="Create clear, well-structured, concise answers from shared information."
)

critic = make_agent(
    role="Critic",
    description="Review quality, accuracy, clarity. Suggest improvements or approve."
)

# ─── 9. Router with safety ───────────────────────────────────────────────────
def route(state: AgentState) -> str:
    if state.get("tool_use_count", 0) >= 7:
        return "writer"  # force exit from endless research

    nxt = state.get("next", "END").strip().lower()
    if nxt in ["researcher", "writer", "critic"]:
        return nxt
    return END

# ─── 10. Build graph ─────────────────────────────────────────────────────────
workflow = StateGraph(state_schema=AgentState)

workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "researcher")

for node in ["researcher", "writer", "critic"]:
    workflow.add_conditional_edges(
        node,
        route,
        {"researcher": "researcher", "writer": "writer", "critic": "critic", END: END}
    )

workflow.add_edge("tools", "researcher")  # most tools used by researcher

graph = workflow.compile()

# ─── 11. Interactive loop ────────────────────────────────────────────────────
print("\n" + "═"*80)
print("  Multi-Agent mesh chat  (type 'exit' / 'quit' / empty line to stop)")
print("═"*80 + "\n")

history: List[BaseMessage] = []

while True:
    q = input("You: ").strip()

    if not q or q.lower() in ["exit", "quit", "q"]:
        print("\nBye! ✌️")
        break

    clear_output(wait=True)
    print("\n" + "═"*80)
    print(f"Query: {q}")
    print("═"*80 + "\n")

    try:
        result = graph.invoke(
            {
                "messages": history + [HumanMessage(content=q)],
                "next": "researcher",
                "tool_use_count": 0
            },
            config={"recursion_limit": 24}   # hard safety limit
        )

        history = result["messages"]

        # Show final answer
        last = result["messages"][-1]
        display(Markdown(f"**Final answer**\n\n{last.content.strip()}\n"))

    except GraphRecursionError:
        display(Markdown(
            "**Warning** — maximum steps reached (possible loop).\n"
            "Showing last message:"
        ))
        if "messages" in result:
            display(Markdown(result["messages"][-1].content))

    # ── Trace last messages ──────────────────────────────────────────────────
    print("\nRecent messages (debug trace):")
    for m in result["messages"][-6:]:
        if isinstance(m, HumanMessage):
            print(f"You: {m.content[:70]}...")
        elif isinstance(m, AIMessage):
            prefix = "🔧 " if m.tool_calls else "→ "
            print(f"{prefix}{m.content[:70]}...")
    print("═"*80)


════════════════════════════════════════════════════════════════════════════════
Query: hi
════════════════════════════════════════════════════════════════════════════════



KeyboardInterrupt: 

In [6]:
# Simple Mesh-style Multi-Agent Collaboration with Groq + LangGraph
# No external tools — pure LLM agents deciding who works next
# Google Colab friendly — January 2026

# ─── Install packages ────────────────────────────────────────────────────────
!pip install -q -U langgraph langchain langchain-groq

# ─── Imports ─────────────────────────────────────────────────────────────────
from typing import Annotated, TypedDict, List
from IPython.display import display, Markdown, clear_output

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, START, END

# ─── LLM Setup (Groq) ────────────────────────────────────────────────────────
llm = ChatGroq(
    model="llama-3.3-70b-versatile",   # or "mixtral-8x7b-32768" etc.
    temperature=0.65,
    max_tokens=1200,
    max_retries=2
)

# ─── State ───────────────────────────────────────────────────────────────────
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "shared conversation history"]
    next_agent: str                     # who should work next


# ─── Agent node factory ──────────────────────────────────────────────────────
def create_agent_node(name: str, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt + """

=== VERY STRICT INSTRUCTION - MUST FOLLOW EXACTLY ===

After you finish your complete response you **MUST** output **EXACTLY** this line and nothing else after it:

next_agent: FINISH

or one of these:

next_agent: planner
next_agent: writer
next_agent: critic

Rules:
- The line must be exactly "next_agent: " followed by one of the four words
- No extra spaces before "next_agent"
- No punctuation, no extra text, no explanation after the line
- Put this line as the VERY LAST line of your entire response
- Only use FINISH when you believe the task is complete and good enough
        """),
        MessagesPlaceholder("messages"),
    ])

    chain = prompt | llm

    def node(state: AgentState):
        messages = state["messages"]
        try:
            response = chain.invoke({"messages": messages})
            content = response.content.strip()

            # More robust parsing - look for pattern anywhere but prefer end
            import re
            match = re.search(r'next_agent:\s*(\w+)(?:\s|$)', content, re.IGNORECASE)
            next_agent = "FINISH"

            if match:
                candidate = match.group(1).strip().lower()
                if candidate in ["planner", "writer", "critic", "finish"]:
                    next_agent = candidate
                    # Try to remove the matched line
                    content = re.sub(r'next_agent:\s*\w+\s*$', '', content, flags=re.IGNORECASE).rstrip()

            return {
                "messages": messages + [AIMessage(content=content, name=name)],
                "next_agent": next_agent
            }
        except Exception as e:
            # Emergency fallback - don't crash the whole flow
            return {
                "messages": messages + [AIMessage(content=f"[Agent failed to respond properly]", name=name)],
                "next_agent": "FINISH"
            }

    return node


# ─── Create three agents with different roles ────────────────────────────────
planner = create_agent_node(
    name="Planner",
    system_prompt="You are a PLANNING expert. Create clear step-by-step plans, think about structure and logic."
)

writer = create_agent_node(
    name="Writer",
    system_prompt="You are a professional WRITER. Write clear, well-structured, engaging text based on plans or previous content."
)

critic = create_agent_node(
    name="Critic",
    system_prompt="You are a sharp CRITIC. Evaluate quality, clarity, completeness, and suggest improvements or approve."
)


# ─── Simple router ───────────────────────────────────────────────────────────
def router(state: AgentState) -> str:
    nxt = state["next_agent"].strip()
    if nxt in ["planner", "writer", "critic"]:
        return nxt
    return END


# ─── Build the graph ─────────────────────────────────────────────────────────
workflow = StateGraph(state_schema=AgentState)

workflow.add_node("planner", planner)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)

# Start always with planner
workflow.add_edge(START, "planner")

# Conditional routing after each agent
for node_name in ["planner", "writer", "critic"]:
    workflow.add_conditional_edges(
        node_name,
        router,
        {
            "planner": "planner",
            "writer": "writer",
            "critic": "critic",
            END: END
        }
    )

# Compile the graph
graph = workflow.compile()


# ─── Interactive Colab interface ─────────────────────────────────────────────
print("Mesh-style multi-agent demo using Groq")
print("Agents: Planner → Writer → Critic")
print("Type your request. Empty line or 'exit' to quit.\n")

history: List[BaseMessage] = []

while True:
    query = input("You: ").strip()

    if not query or query.lower() in ["exit", "quit", "q"]:
        print("\nGoodbye!")
        break

    clear_output(wait=True)
    print("\n" + "═" * 80)
    print(f"Query: {query}")
    print("═" * 80 + "\n")

    # Run the graph
    try:
        result = graph.invoke({
            "messages": history + [HumanMessage(content=query)],
            "next_agent": "planner"
        }, {"recursion_limit": 20})

        history = result["messages"]

        # Show final answer
        final_msg = result["messages"][-1]
        display(Markdown(f"**Final output**\n\n{final_msg.content.strip()}\n"))

        # Short trace
        print("\nTrace (last 4 messages):")
        for msg in result["messages"][-4:]:
            prefix = "You: " if isinstance(msg, HumanMessage) else f"{msg.name or 'Agent'}: "
            print(prefix + msg.content[:70] + ("..." if len(msg.content) > 70 else ""))
        print("═" * 80)

    except Exception as e:
        print(f"Error during execution: {str(e)}")
        print("Try again with a different query or increase recursion limit.")


════════════════════════════════════════════════════════════════════════════════
Query: explain quantum rl
════════════════════════════════════════════════════════════════════════════════



**Final output**

Researchers are working on developing practical applications of QRL.



Trace (last 4 messages):
You: explain quantum rl
Planner: Quantum Reinforcement Learning (QRL) is a subfield of machine learning...
Planner: Researchers are working on developing practical applications of QRL.
════════════════════════════════════════════════════════════════════════════════


KeyboardInterrupt: Interrupted by user